In [1]:
!pip install spacy
!pip install minibatch
!pip install compounding
!pip install Path

In [2]:
#ERROR: For Cory, find out why update() function does not work in spacy's Named Entity Recognizer.


!pip install -U pip setuptools wheel
#!pip install -U 'spacy[cuda-autodetect]'
!python -m spacy download en_core_web_sm
#Load pre-existing model.

import spacy
import random
from spacy.util import minibatch, compounding
from pathlib import Path

nlp = spacy.load('en_core_web_sm')

#Construct pipeline component.

ner = nlp.get_pipe("ner")

#Training data to update model.

train_data = [
    ("I have terminal diabetes we do not do sarcasm", {"entities":[(16,24, "DIABETES")]}),
    ("I do actually I have type one diabetes", {"entities":[(21,37, "TYPE 1 DIABETES")]}),
    ("Making meal plan ideas at work I am trying to reverse my a1c within six months", {"entities":[(57,60, "BLOOD SUGAR")]}),
    ("I think I got Diabetes I am too young", {"entities":[(14,22, "DIABETES")]}),
    ("I am injecting myself with insulin everyday", {"entities":[(27,34, "BLOOD SUGAR")]}),
    ("The doctor said I was nowhere near pre diabetes my numbers are normal", {"entities":[(35,47, "PREDIABETES")]}),
    ("People with high A1C test so they do not get full blown diabetes", {"entities":[(17,20, "BLOOD SUGAR")]}),
    ("I have abnormally low blood sugar", {"entities":[(18,32, "TYPE I DIABETES")]}),
    ("Looks like he has type two diabetes", {"entities":[(18,34, "TYPE II DIABETES")]}),
    ("I forgot about type 2 diabetes", {"entities": [(15,29, "TYPE II DIABETES")]}),
    ("Type 1 diabetes is not to be underestimated", {"entities": [(0,15, "TYPE I DIABETES")]}),
    ("My blood sugar levels are too unstable", {"entities": [(3,13, "BLOOD SUGAR")]})
]

#Add labels to the NER model.

for _, modifications in train_data:
    for entit in modifications.get("entities"):
        ner.add_label(entit[2])
        
#Disable pipeline parts that dont need to be modified.

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

#Model Training

with nlp.disable_pipes(*unaffected_pipes):
    for i in range(30):
        random.shuffle(train_data)
        losses = {}
        
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                        texts,
                        annotations,
                        drop = 0.5,
                        losses=losses
            )
            print("Losses", losses)
            
#Model testing

doc = nlp("I have diabetes")
print("Entities", [(entit.text, entit.label_) for entit in doc.ents])

ERROR: To modify pip, please run the following command:
C:\Users\coryg\anaconda3\python.exe -m pip install -U pip setuptools wheel


  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/8a/6a/19e9fe04fca059ccf770861c7d5721ab4c2aebc539889e97c7977528a53b/pip-24.0-py3-none-any.whl.metadata
  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
  Obtaining dependency information for setuptools from https://files.pythonhosted.org/packages/f7/29/13965af254e3373bceae8fb9a0e6ea0d0e571171b80d6646932131d6439b/setuptools-69.5.1-py3-none-any.whl.metadata
  Using cached setuptools-69.5.1-py3-none-any.whl.metadata (6.2 kB)
  Obtaining dependency information for wheel from https://files.pythonhosted.org/packages/7d/cd/d7460c9a869b16c3dd4e1e403cce337df165368c71d6af229a74699622ce/wheel-0.43.0-py3-none-any.whl.metadata
  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)
Using cached setuptools-69.5.1-py3-none-any.whl (894 kB)
Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
     ---------------------------------------- 0.0/12.8 MB ? et

ValueError: [E989] `nlp.update()` was called with two positional arguments. This may be due to a backwards-incompatible change to the format of the training data in spaCy 3.0 onwards. The 'update' function should now be called with a batch of Example objects, instead of `(text, annotation)` tuples. 